<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Risk Parity & Budgeting with Python

### Mean-Variance Optimal Portfolio

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | [team@tpq.io](mailto:team@tpq.io) 

<img src="http://hilpisch.com/images/py4fi_2nd.png" width="20%" align="left">

## Financial Assets

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
np.set_printoptions(suppress=True)
%config InlineBackend.figure_format = 'svg'

In [ ]:
url = 'universe.csv'

In [ ]:
data = pd.read_csv(url, index_col=0, parse_dates=True)
data = data.loc['2019-1-1':]

In [ ]:
data.info()

In [ ]:
data.head()

### Returns

In [ ]:
rets = np.log(data / data.shift(1))  # daily log returns
rets.head()

In [ ]:
rets.hist(figsize=(10, 6), bins=35);

In [ ]:
np.sqrt(rets.var() * 252)  # annualized volatility

## Portfolio Return

In [ ]:
noa = data.shape[1]
noa

In [ ]:
phi = np.ones(noa) / noa  # equally weighted portfolio
phi

In [ ]:
plt.pie(phi, labels=data.columns, autopct='%1.1f%%');

In [ ]:
def port_return(phi):
    return np.dot(rets.mean() * 252, phi)

In [ ]:
port_return(phi)

## Portfolio Risk

In [ ]:
def port_risk(phi):
    return math.sqrt(np.dot(phi, np.dot(rets.cov() * 252, phi)))

In [ ]:
port_risk(phi)

In [ ]:
rets.corr()

## Maximum Sharpe Portfolio

In [ ]:
from scipy.optimize import minimize

In [ ]:
def sharpe(phi):
    return port_return(phi) / port_risk(phi)

In [ ]:
sharpe(phi)

In [ ]:
bnds = noa * [(0, 1)]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: phi.sum() - 1}

In [ ]:
opt = minimize(lambda phi: -sharpe(phi), phi,
               bounds=bnds,
               constraints=cons)
opt

In [ ]:
phi_ = opt['x']
phi_

In [ ]:
plt.pie(phi_, labels=data.columns, autopct='%1.1f%%');

In [ ]:
sharpe(phi_)  # maximum Sharpe ratio

In [ ]:
port_return(phi_)

In [ ]:
port_risk(phi_)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>